In [27]:
import os
import scipy
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import multivelo as mv
import matplotlib.pyplot as plt
from pathlib import Path
import utils
import gc

In [2]:
scv.settings.verbosity = 3
scv.settings.presenter_view = True
scv.set_figure_params('scvelo')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
np.set_printoptions(suppress=True)
DATA_DIR = Path("/root/autodl-tmp/dataset/ATAC")
DATASET = "mouse_brain"

In [28]:
adata_rna = sc.read(DATA_DIR / DATASET / "raw" / "10X_multiome_mouse_brain.loom", cache=True)
adata_rna.obs_names = [x.split(':')[1][:-1] + '-1' for x in adata_rna.obs_names]
adata_rna.var_names_make_unique()

In [29]:
sc.pp.filter_cells(adata_rna, min_counts=1000)
sc.pp.filter_cells(adata_rna, max_counts=20000)

In [30]:
scv.pp.filter_and_normalize(adata_rna, min_shared_counts=10, n_top_genes=1000)

Filtered out 21580 genes that are detected 10 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 1000 highly variable genes.
Logarithmized X.


/root/autodl-tmp/conda/envs/atac_py310/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


In [31]:
cell_annot = pd.read_csv(DATA_DIR / DATASET / "raw" / "cell_annotations.tsv", sep='\t', index_col=0)

In [32]:
adata_rna = adata_rna[cell_annot.index,:]
adata_rna.obs['celltype'] = cell_annot['celltype']

In [33]:
adata_rna = adata_rna[adata_rna.obs['celltype'].isin(['RG, Astro, OPC',
                                                      'IPC',
                                                      'V-SVZ',
                                                      'Upper Layer',
                                                      'Deeper Layer',
                                                      'Ependymal cells',
                                                      'Subplate'])]

In [34]:
adata_atac = sc.read_10x_mtx(DATA_DIR / DATASET / "raw" / "filtered_feature_bc_matrix/", var_names='gene_symbols', cache=True, gex_only=False)
adata_atac = adata_atac[:,adata_atac.var['feature_types'] == "Peaks"]

In [35]:
adata_atac = mv.aggregate_peaks_10x(adata_atac,
                                    DATA_DIR / DATASET / "raw" / "peak_annotation.tsv",
                                    DATA_DIR / DATASET / "raw" / "analysis/feature_linkage/feature_linkage.bedpe")

CellRanger ARC identified as 1.0.0

Found 19006 genes with promoter peaks



/root/autodl-tmp/conda/envs/atac_py310/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/19006 [00:00<?, ?it/s]

In [36]:
sc.pp.filter_cells(adata_atac, min_counts=2000)
sc.pp.filter_cells(adata_atac, max_counts=60000)

In [37]:
mv.tfidf_norm(adata_atac)

In [38]:
shared_cells = pd.Index(np.intersect1d(adata_rna.obs_names, adata_atac.obs_names))
shared_genes = pd.Index(np.intersect1d(adata_rna.var_names, adata_atac.var_names))
len(shared_cells), len(shared_genes)

(3365, 936)

## Reload the spliced and unspliced matrices

In [39]:
adata_rna = sc.read(DATA_DIR / DATASET / "raw" / "10X_multiome_mouse_brain.loom", cache=True)
adata_rna.obs_names = [x.split(':')[1][:-1] + '-1' for x in adata_rna.obs_names]
adata_rna.var_names_make_unique()

In [40]:
adata_rna = adata_rna[shared_cells, shared_genes]
adata_atac = adata_atac[shared_cells, shared_genes]

## Begin preprocessing adata_rna

In [41]:
adata_rna.obs['celltype'] = cell_annot.loc[adata_rna.obs_names, 'celltype']
adata_rna.obs['celltype'] = adata_rna.obs['celltype'].astype('category')

In [42]:
all_clusters = ['Upper Layer',
                'Deeper Layer',
                'V-SVZ',
                'RG, Astro, OPC',
                'Ependymal cells',
                'IPC',
                'Subplate']
adata_rna.obs['celltype'] = adata_rna.obs['celltype'].cat.reorder_categories(all_clusters)

## preprocess adata_atac

In [ ]:
adata_rna.obs_names.to_frame().to_csv(DATA_DIR / DATASET / "raw" / "seurat_wnn" / "filtered_cells.txt", header=False, index=False)

In [ ]:
# Read in Seurat WNN neighbors.
nn_idx = np.loadtxt(DATA_DIR / DATASET / "raw" / "seurat_wnn" / "nn_idx.txt", delimiter=',')
nn_dist = np.loadtxt(DATA_DIR / DATASET / "raw" / "seurat_wnn" / "nn_dist.txt", delimiter=',')
nn_cells = pd.Index(pd.read_csv(DATA_DIR / DATASET / "raw" / "seurat_wnn" / "nn_cells.txt", header=None)[0])
# Make sure cell names match.
np.all(nn_cells == adata_atac.obs_names)

In [ ]:
mv.knn_smooth_chrom(adata_atac, nn_idx, nn_dist)

## split and preprocess

In [ ]:
K_FOLD = 3
CLUSTER_KEY = "celltype"
SAVE_DATA = True

In [ ]:
sub_adata_lst = utils.split_anndata_stratified(adata_rna, 
                                               adata_atac,
                                               n_splits=K_FOLD, 
                                               cluster_key=CLUSTER_KEY)

In [ ]:
for i in range(len(sub_adata_lst)):
    sub_adata, sub_adata_atac = sub_adata_lst[i]
    print("check 1: ", sub_adata.shape == sub_adata_atac.shape)
    sub_adata.layers['raw_spliced'] = sub_adata.layers['spliced'].copy()
    sub_adata.layers['raw_unspliced'] = sub_adata.layers['unspliced'].copy()
    scv.pp.normalize_per_cell(sub_adata)
    scv.pp.log1p(sub_adata)
    sc.pp.highly_variable_genes(sub_adata, n_top_genes=sub_adata.n_vars, subset=False)
    print("check 2: ", sub_adata.shape == sub_adata_atac.shape)
    if hasattr(sub_adata, "obsm"):
        del sub_adata.obsm
    if hasattr(sub_adata, "obsp"):
        del sub_adata.obsp
    if hasattr(sub_adata, "varm"):
        del sub_adata.varm
    if 'pca' in sub_adata.uns:
        del sub_adata.uns['pca']
    if 'umap' in sub_adata.uns:
        del sub_adata.uns['umap']
    if "neighbors" in sub_adata.uns:
        del sub_adata.uns['neighbors']
    if 'Ms' in sub_adata.layers:
        del sub_adata.layers['Ms']
    if 'Mu' in sub_adata.layers:
        del sub_adata.layers['Mu']
    scv.pp.moments(sub_adata, n_neighbors=30, n_pcs=30)
    print("check 3: ", sub_adata.shape == sub_adata_atac.shape)
    utils.fill_in_neighbors_indices(sub_adata)
    sc.tl.umap(sub_adata)
    sub_adata.obs['u_lib_size_raw'] = sub_adata.layers['raw_unspliced'].toarray().sum(-1) 
    sub_adata.obs['s_lib_size_raw'] = sub_adata.layers['raw_spliced'].toarray().sum(-1)
    scv.pl.umap(sub_adata, color=CLUSTER_KEY)
    print("check 4: ", sub_adata.shape == sub_adata_atac.shape)
    if SAVE_DATA:
        (DATA_DIR / DATASET / "processed").mkdir(parents=True, exist_ok=True)
        sub_adata.write_h5ad(DATA_DIR / DATASET / "processed" / f"adata_preprocessed_{i}.h5ad")
        sub_adata_atac.write_h5ad(DATA_DIR / DATASET / "processed" / f"adata_atac_preprocessed_{i}.h5ad")
    del sub_adata, sub_adata_atac
    gc.collect()